In [1]:
data <- read.csv("Data_T3.csv")

In [2]:
# a first look at the data
head(data)
summary(data)

,X,ID,TreatmentGroup,Age,Sex,ECOG_PS,GFR,Time,Event
,<int>,<int>,<chr>,<int>,<chr>,<int>,<int>,<dbl>,<int>
1,1,1,Treatment,73,Female,2,83,0.6,1
2,2,2,Treatment,62,Male,0,86,6.3,1
3,3,3,Treatment,61,Female,0,98,57.7,0
4,4,4,Treatment,66,Male,1,58,0.7,1
5,5,5,Treatment,88,Female,0,120,30.7,0
6,6,6,Treatment,73,Male,0,55,6.5,1


       X                ID         TreatmentGroup          Age      
 Min.   :  1.00   Min.   :  1.00   Length:300         Min.   :38.0  
 1st Qu.: 75.75   1st Qu.: 75.75   Class :character   1st Qu.:59.0  
 Median :150.50   Median :150.50   Mode  :character   Median :65.0  
 Mean   :150.50   Mean   :150.50                      Mean   :65.3  
 3rd Qu.:225.25   3rd Qu.:225.25                      3rd Qu.:72.0  
 Max.   :300.00   Max.   :300.00                      Max.   :91.0  
     Sex               ECOG_PS            GFR              Time       
 Length:300         Min.   :0.0000   Min.   : 30.00   Min.   : 0.000  
 Class :character   1st Qu.:0.0000   1st Qu.: 66.00   1st Qu.: 2.600  
 Mode  :character   Median :1.0000   Median : 81.00   Median : 5.900  
                    Mean   :0.8167   Mean   : 78.89   Mean   : 9.673  
                    3rd Qu.:1.0000   3rd Qu.: 91.00   3rd Qu.:13.600  
                    Max.   :2.0000   Max.   :120.00   Max.   :57.700  
     Event     
 Min